# Cross-validation version

In [1]:
import math
import random
import numpy as np
np.random.seed(33)

## Initializing spark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("appName").setMaster("local")
sc = SparkContext()

## Functions

### Auxiliar functions

In [4]:
def trainTestSplit(dataset):
    train = dataset.filter(lambda x: x[2]==1)
    train = train.map(lambda x: (x[0], x[1]))
    test = dataset.filter(lambda x: x[2]==0)
    test = test.map(lambda x: (x[0], x[1]))
    return (train, test)

In [5]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [6]:
def rdd_cost_function(RDD_Xyyhat, lambda_ref, m, w):
    y = RDD_Xyyhat[1]
    y_hat = RDD_Xyyhat[2]
    temp = (-1/m) * (
        y * np.log(y_hat) + (1 - y)
        * np.log(1 - y_hat)
    )
    cost = temp + (lambda_ref/(2*m))*sum([i*i for i in w.value])
    return cost

In [7]:
def str_to_number_list(line):
    numbers = line.replace(" ","").split(",")
    numbers = [float(number) for number in numbers]
    
    # Separate X and y
    numbers = (numbers[:-1], numbers[-1])
    
    return numbers

In [8]:
def sum_list_values(list1, list2):
    return [n1 + n2 for n1, n2 in zip(list1, list2)]

In [9]:
def sum_minus_mean_squared(numbers):
    return [(number-means.value[x])**2 for x,number in enumerate(numbers)]

In [10]:
def rdd_mean_by_column(RDD_Xy, m):
    means = RDD_Xy.map(lambda x: x[0]).reduce(sum_list_values)
    means = [x/m for x in means]
    return means

In [11]:
def rdd_std_by_column(RDD_Xy, m):
    stds = RDD_Xy.map(lambda x: x[0]).map(sum_minus_mean_squared).reduce(sum_list_values)
    stds = [math.sqrt(x/m) for x in stds]
    return stds

In [12]:
def multiply_RDDXy_by_w(Xy):
    tot = 0
    for xi, wi in zip(Xy[0], w.value):
        tot += xi * wi
    return (Xy[0], Xy[1], sigmoid(tot))

In [13]:
def calculate_dw(RDD_Xyyhat):
    dw = []
    for x in RDD_Xyyhat[0]:
        dw.append((RDD_Xyyhat[1]-RDD_Xyyhat[2])*x)
    return dw

### Mandatory functions

In [48]:
def readFile(filename):
    dataset = sc.textFile(filename)
    dataset = dataset.map(str_to_number_list)
    return dataset.sample(False, 0.00012)

In [15]:
def normalize(RDD_Xy):
    m = RDD_Xy.count()
    global means
    means = sc.broadcast(rdd_mean_by_column(RDD_Xy, m))
    stds = sc.broadcast(rdd_std_by_column(RDD_Xy, m))
    norm_rdd = RDD_Xy.map(lambda values: ([(x - mean) / std for x, mean, std in zip(values[0], means.value, stds.value)], values[1]))
    return norm_rdd

In [16]:
def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    global b
    global m
    m = RDD_Xy.count()
    n = len(RDD_Xy.take(1)[0])
    
    global w
    w = sc.broadcast(np.random.rand(n))
    
    
    for it in range(iterations):
        RDD_Xyyhat = RDD_Xy.map(multiply_RDDXy_by_w)
        print(f"Cost for it {it}:", RDD_Xyyhat.map(lambda x: rdd_cost_function(x, lambda_reg, m, w)).reduce(lambda x,y: x+y))
        dw = RDD_Xyyhat.map(calculate_dw).reduce(lambda x, y: [xi+yi for xi, yi in zip(x, y)])        
        dw_final = []
        for dwi, wi in zip(dw, w.value):
            dw_final.append( ((dwi+lambda_reg)/m)+wi )
        w = sc.broadcast([wi - (learning_rate * dwi) for wi, dwi in zip(w.value, dw_final)])
        
        b -= learning_rate * (1/m)*RDD_Xyyhat.map(lambda x: x[1]-x[2]).sum()
    
    return w, b


In [17]:
def accuracy(ws, b, RDD_Xy):
    total = RDD_Xy.count()
    y_and_y_hat = RDD_Xy.map(lambda x: (x[1], predict(ws, x[0], b)))
    correct = y_and_y_hat.map(lambda x: 1 if x[0]==x[1] else 0).reduce(lambda x, y: x+y)
    return correct/total

In [18]:
def predict(w, X, b):
    tot = 0
    for xi, wi in zip(X, w.value):
        tot += xi * wi
    tot += b
    val = sigmoid(tot)
    if (val>=0.5):
        return (1.0)
    return (0.0)

### Make folds for cross-validation

In [25]:
def shuffleRDD(RDD):
    RDD_with_rand = RDD.map(lambda x: (x[0], x[1], random.random()))
    RDD_sorted = RDD_with_rand.sortBy(lambda x: x[2])
    RDD_cleaned = RDD_sorted.map(lambda x: (x[0], x[1]))
    return RDD_cleaned

In [36]:
def kFoldsCV(k_fold, iterations, rdd_data, learning_rate, lambda_reg):
    
    n = len(rdd_data.take(1)[0])
    
    fold_length = rdd_data.count()/k_fold
    train_errors_fold = []
    test_errors_fold = []
    folds_accuracy = []
    
    for i_fold in range(k_fold):
        w = sc.broadcast(np.random.rand(n))
        
        global b
        b = 0
        
        starting_fold = fold_length * i_fold
        end_fold = starting_fold + fold_length
        
        train_fold = rdd_data.zipWithIndex().filter(lambda x: (x[1] >= starting_fold and x[1]<end_fold)).map(lambda x: x[0])
        test_fold = rdd_data.zipWithIndex().filter(lambda x: (x[1] < starting_fold or x[1] >= end_fold)).map(lambda x: x[0])
        train_size = train_fold.count()
        
        w, b = train(train_fold, iterations, learning_rate, lambda_reg) 
        fold_accuracy = accuracy(w, b, test_fold)
        folds_accuracy.append(fold_accuracy)
        print(f"Accuracy for fold {i_fold}: {fold_accuracy}")
    
    print(folds_accuracy)
    return sum(folds_accuracy)/len(folds_accuracy)

## Testing

In [49]:
RDD_Xy = readFile("../data/botnet_tot_syn_l.csv")
RDD_Xy.count()

110

In [50]:
RDD_Xy_normalized = normalize(RDD_Xy)

In [51]:
RDD_shuffled = shuffleRDD(RDD_Xy_normalized)

In [53]:
avg_accuracy = kFoldsCV(10, 10, RDD_shuffled, 0.3, 0.1)
avg_accuracy

Cost for it 0: 1.1383892566641054
Cost for it 1: 1.0509826069338688
Cost for it 2: 0.9902484196262833
Cost for it 3: 0.9472970266779381
Cost for it 4: 0.9164432968165312
Cost for it 5: 0.8939785313569335
Cost for it 6: 0.8774337584463693
Cost for it 7: 0.865132358048245
Cost for it 8: 0.8559141619273699
Cost for it 9: 0.8489623372689276
Accuracy for fold 0: 0.46464646464646464
Cost for it 0: 0.725218704328911
Cost for it 1: 0.6967606960146799
Cost for it 2: 0.6848492996729586
Cost for it 3: 0.6820273270085991
Cost for it 4: 0.6839343408230244
Cost for it 5: 0.6880498713586367
Cost for it 6: 0.6929510313617667
Cost for it 7: 0.6978678346044869
Cost for it 8: 0.7024144622869745
Cost for it 9: 0.7064265165685323
Accuracy for fold 1: 0.5151515151515151
Cost for it 0: 0.7571803351740949
Cost for it 1: 0.7704702435473203
Cost for it 2: 0.7808939316948097
Cost for it 3: 0.7889904446696898
Cost for it 4: 0.795236819338879
Cost for it 5: 0.8000329917671662
Cost for it 6: 0.8037034669339149
Cost

0.4616161616161617